# Retail Demand Forecasting (Weekly Store×SKU)

Baseline vs ML forecasting with time features.


In [ ]:
import os
from pathlib import Path

def find_project_root(start: Path, marker: str = "02_retail_demand_forecasting") -> Path:
    p = start.resolve()
    for parent in [p] + list(p.parents):
        if parent.name == marker:
            return parent
    return start.resolve()

ROOT = find_project_root(Path.cwd())
os.chdir(ROOT)
print("Project root:", ROOT)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor


In [ ]:
data_path = Path("data/weekly_demand.csv")
if not data_path.exists():
    from data.make_dataset import main as make_data
    make_data(out_path=str(data_path))

df = pd.read_csv(data_path, parse_dates=["week_start"])
df.head()


In [ ]:
df["weekofyear"] = df["week_start"].dt.isocalendar().week.astype(int)
df["month"] = df["week_start"].dt.month
df["year"] = df["week_start"].dt.year
df["sin_woy"] = np.sin(2*np.pi*df["weekofyear"]/52.0)
df["cos_woy"] = np.cos(2*np.pi*df["weekofyear"]/52.0)

cutoff = df["week_start"].quantile(0.8)
train = df[df["week_start"] <= cutoff].copy()
test = df[df["week_start"] > cutoff].copy()

X_train = train.drop(columns=["units"])
y_train = train["units"]
X_test = test.drop(columns=["units"])
y_test = test["units"]


In [ ]:
# Baseline: median units (simple, but gives a sanity check)
baseline = pd.Series(train["units"].median(), index=test.index)
print("Baseline MAE:", round(mean_absolute_error(y_test, baseline), 2))


In [ ]:
cat = ["store_id","sku_id"]
num = ["price","promo","holiday","weekofyear","month","year","sin_woy","cos_woy"]

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat),
    ("num", "passthrough", num)
])

model = Pipeline([
    ("prep", pre),
    ("model", HistGradientBoostingRegressor(
        max_depth=8, learning_rate=0.08, max_iter=300, random_state=42
    ))
])

model.fit(X_train, y_train)
pred = model.predict(X_test)
mae = mean_absolute_error(y_test, pred)
print("GBR MAE:", round(mae, 2))


In [ ]:
# Example plot
sample = test[(test["store_id"]=="S100") & (test["sku_id"]=="SKU1000")].sort_values("week_start")
if len(sample) > 0:
    plt.figure(figsize=(9,4))
    plt.plot(sample["week_start"], sample["units"], label="Actual")
    plt.plot(sample["week_start"], model.predict(sample.drop(columns=["units"])), label="Forecast")
    plt.title("Example Forecast — S100 × SKU1000")
    plt.xlabel("Week"); plt.ylabel("Units")
    plt.legend()
    plt.tight_layout()

    Path("reports").mkdir(exist_ok=True)
    plt.savefig("reports/example_forecast.png", dpi=200, bbox_inches="tight")
    plt.show()


In [ ]:
# Export artifacts
import joblib
Path("models").mkdir(exist_ok=True)
Path("reports").mkdir(exist_ok=True)

joblib.dump({"model": model, "mae": float(mae)}, "models/demand_forecaster.joblib")
Path("reports/metrics.json").write_text(pd.Series({"mae": float(mae)}).to_json(), encoding="utf-8")
print("Saved models/demand_forecaster.joblib and reports/*")


## Recommendations
- Use the ML forecaster for replenishment.
- Translate MAE into safety stock by SKU (higher MAE → larger buffer).
- Monitor promo/holiday effects and retrain regularly.
